In [1]:
import pandas as pd
from pandas import DataFrame
df=pd.read_csv("A1.csv",names=['id','label'])
df1_T=df[0:49]
df1_V=df[49:74]
training_set = pd.read_csv('A1.csv').values

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
train_generator = train_datagen.flow_from_dataframe(
        dataframe=df,
        directory='A1_LCC/A1_T',
        x_col="id",
        y_col="label",
        shuffle=True,
        target_size=(320, 320),
        batch_size=6,
        class_mode="raw")
valid_generator=train_datagen.flow_from_dataframe(
                dataframe=df1_V,
                directory='A1_LCC/A1_V',
                x_col="id",
                y_col="label",
               
                batch_size=32,
                #seed=49,
                shuffle=True,
                class_mode="raw",
                target_size=(320,320))

Found 49 validated image filenames.
Found 25 validated image filenames.


/usr/local/lib/python3.5/dist-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 79 invalid image filename(s) in x_col="id". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [21]:
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras import regularizers
base_model=MobileNet(input_shape=(320,320,3),weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(1,activation='relu')(x) #final layer with softmax activation

/usr/local/lib/python3.5/dist-packages/keras_applications/mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [22]:
from keras.models import Model
model=Model(inputs=base_model.input,outputs=preds)

In [23]:
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [24]:
import keras
from keras.callbacks import EarlyStopping
csv_logger = keras.callbacks.CSVLogger('training.log', separator=',')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.03, mode='min', patience=8)

In [25]:
from keras.optimizers import RMSprop
step_size_train=train_generator.n//train_generator.batch_size
model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'mse', 
              metrics = ['accuracy'])
#model.fit_generator(generator=train_generator,
 #                  steps_per_epoch=step_size_train,
  #                 epochs=50)
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size
model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    validation_data=valid_generator,
                    validation_steps=step_size_valid,
                    #steps_per_epoch=1,
                    epochs=50,
                    callbacks= [csv_logger,early_stop])

Epoch 1/50
8/8 [==============================] - 7s 885ms/step - loss: 80.4037 - accuracy: 0.1395 - val_loss: 41.6455 - val_accuracy: 0.0000e+00
Epoch 2/50
8/8 [==============================] - 2s 232ms/step - loss: 1.6720 - accuracy: 0.3488 - val_loss: 24.7717 - val_accuracy: 0.0000e+00
Epoch 3/50
8/8 [==============================] - 2s 241ms/step - loss: 4.2411 - accuracy: 0.1860 - val_loss: 11.9638 - val_accuracy: 0.0000e+00
Epoch 4/50
8/8 [==============================] - 2s 247ms/step - loss: 4.1628 - accuracy: 0.2791 - val_loss: 17.5951 - val_accuracy: 0.0000e+00
Epoch 5/50
8/8 [==============================] - 2s 266ms/step - loss: 1.9220 - accuracy: 0.2093 - val_loss: 2.8387 - val_accuracy: 0.2000
Epoch 6/50
8/8 [==============================] - 2s 237ms/step - loss: 5.7743 - accuracy: 0.0465 - val_loss: 6.2729 - val_accuracy: 0.0800
Epoch 7/50
8/8 [==============================] - 2s 266ms/step - loss: 2.8361 - accuracy: 0.2917 - val_loss: 1.7610 - val_accuracy: 0.3200